# Data Cleaning

## Movies file cleaning
### Counting all the movies in the file

In [20]:
import pandas as pd

movies_df = pd.read_csv('movies.csv')
print('Total number of movies before cleaning: ', len(movies_df))

Total number of movies before cleaning:  941597


### Cleaning empty name, rating and date columns

In [29]:
movies_cleaned=movies_df

#remove empty strings or null values from name column
movies_cleaned = movies_cleaned.dropna(subset=['name'])
print('Cleaning empty name values: ', len(movies_cleaned))

#remove empty strings or null values from rating column
movies_cleaned = movies_cleaned.dropna(subset=['rating'])
print('Cleaning empty rating values:', len(movies_cleaned))

#remove empty strings or null values from date column
movies_cleaned = movies_cleaned.dropna(subset=['date'])
print('Cleaning empty date values:', len(movies_cleaned))

Cleaning empty name values:  941587
Cleaning empty rating values: 90999
Cleaning empty date values: 90999


### Cleaning rows with duplicate IDs

In [22]:
#drop duplicate rows with the same id, maintaining the first one
movies_cleaned = movies_cleaned.drop_duplicates(subset=['id'], keep='first')
print('Cleaning duplicate movies: ', len(movies_cleaned),'\n')

#increasing display width to fit all the columns
pd.set_option("display.width", 140)
print('Displaying the cleaned data frame:')

#Displaying shortened columns for the sake of visibility: name, tagline and description have been shortened to 4 words
print(movies_cleaned.assign(tagline=movies_cleaned["tagline"].apply(lambda x: " ".join(str(x).split()[:4])), description=movies_cleaned["description"].apply(lambda x: " ".join(str(x).split()[:4])), name=movies_cleaned["name"].apply(lambda x: " ".join(str(x).split()[:4]))).head().to_string(index=False, col_space=10))

Cleaning duplicate movies:  90999 

Displaying the cleaned data frame:
        id                         name       date                     tagline                       description     minute     rating
   1000001                       Barbie     2023.0 She's everything. He's just                Barbie and Ken are      114.0       3.86
   1000002                     Parasite     2019.0            Act like you own  All unemployed, Ki-taek's family      133.0       4.56
   1000003 Everything Everywhere All at     2022.0          The universe is so        An aging Chinese immigrant      140.0       4.30
   1000004                   Fight Club     1999.0     Mischief. Mayhem. Soap. A ticking-time-bomb insomniac and      139.0       4.27
   1000005                   La La Land     2016.0         Here's to the fools         Mia, an aspiring actress,      129.0       4.09


### Checking data types in the data frame

In [23]:
print(movies_cleaned.dtypes)

id               int64
name            object
date           float64
tagline         object
description     object
minute         float64
rating         float64
dtype: object


### Casting date and minute to avoid the decimal numbers

In [24]:
#casting to integer to avoid showing decimal numbers
movies_cleaned['date']=movies_cleaned['date'].astype('Int64')

#casting to integer to avoid showing decimal numbers
movies_cleaned['minute']=movies_cleaned['minute'].astype('Int64')

#shortened display
print(movies_cleaned.assign(tagline=movies_cleaned["tagline"].apply(lambda x: " ".join(str(x).split()[:4])), description=movies_cleaned["description"].apply(lambda x: " ".join(str(x).split()[:4])), name=movies_cleaned["name"].apply(lambda x: " ".join(str(x).split()[:4]))).head().to_string(index=False, col_space=10))

#generating a new CSV to use during the data analysis
movies_cleaned.to_csv('movies_cleaned.csv', index=False)

        id                         name       date                     tagline                       description     minute     rating
   1000001                       Barbie       2023 She's everything. He's just                Barbie and Ken are        114       3.86
   1000002                     Parasite       2019            Act like you own  All unemployed, Ki-taek's family        133       4.56
   1000003 Everything Everywhere All at       2022          The universe is so        An aging Chinese immigrant        140       4.30
   1000004                   Fight Club       1999     Mischief. Mayhem. Soap. A ticking-time-bomb insomniac and        139       4.27
   1000005                   La La Land       2016         Here's to the fools         Mia, an aspiring actress,        129       4.09


## Genre file cleaning
### Counting all the genres present for each existing movie

In [25]:
genres_df = pd.read_csv('genres.csv')
print('Total number of genres for all present movies before cleaning: ', len(genres_df))

Total number of genres for all present movies before cleaning:  1046849


### Cleaning empty id or genre columns

In [26]:
genres_cleaned=genres_df.dropna(subset=['genre','id'])
print("Cleaning any empty column: ",len(genres_cleaned))

Cleaning any empty column:  1046849


### Removing rows where the same ID has the same genre multiple times

In [27]:
genres_cleaned = genres_cleaned.drop_duplicates(subset=['id','genre'], keep='first')
print("Dropping duplicate genres for the same id: ",len(genres_cleaned),"\n")
print("Displaying the cleaned data frame:")
print(genres_cleaned.head().to_string(index=False, col_space=10))
genres_cleaned.to_csv('genres_cleaned.csv', index=False)

Dropping duplicate genres for the same id:  1046849 

Displaying the cleaned data frame:
        id      genre
   1000001     Comedy
   1000001  Adventure
   1000002     Comedy
   1000002   Thriller
   1000002      Drama


## Crew file cleaning
### Removing duplicate rows and any empty cell

In [17]:
crew_df = pd.read_csv('crew.csv')
print("Amount of known crew members for all the movies considered: ", len(crew_df))
print("Amount with only directors present (and eventual duplicates or unknown names/ids): ", len(crew_df[crew_df["role"]=="Director"]), "\n")

#remove empty strings or null values from role column
crew_cleaned = crew_df.dropna()
print("Cleaning empty cells: ",len(crew_cleaned))
print("Rows with directors:", len(crew_cleaned[crew_cleaned["role"]=="Director"]), "\n")

#apply drop_duplicates only to Directors
directors = (
    crew_cleaned[crew_cleaned["role"] == "Director"]
    .drop_duplicates(subset=["id", "role"], keep="first")
)

#keep all non-directors untouched
others = crew_cleaned[crew_cleaned["role"] != "Director"]

#combine back
crew_cleaned = pd.concat([directors, others]).sort_index()
print('Cleaning duplicate directors: ',len(crew_cleaned))
print("Rows with only directors: ",len(crew_cleaned[crew_cleaned["role"]=="Director"]), "\n")

print('Data preview of crew')
print(crew_cleaned.head().to_string(index=False, col_space=10))
crew_cleaned.to_csv('crew_cleaned.csv', index=False)

Amount of known crew members for all the movies considered:  4720183
Amount with only directors present (and eventual duplicates or unknown names/ids):  900753 

Cleaning empty cells:  4720182
Rows with directors: 900753 

Cleaning duplicate directors:  4612951
Rows with only directors:  793522 

Data preview of crew
        id       role           name
   1000001   Director   Greta Gerwig
   1000001   Producer   Tom Ackerley
   1000001   Producer  Margot Robbie
   1000001   Producer Robbie Brenner
   1000001   Producer   David Heyman
